<b><font size=20, color='#A020F0'>MetPy

Hannah Zanowski<br>
11/18/24<br>

#### <span style="color:green">Learning Goals</span>
By the end of this notebook you will
1. Be able to read in GRIB files using xarray
2. Understand how to use Mety to do unit-aware calculations
3. Use MetPy to calculate and plot various quantities relevant to synoptic meteorology

#### Resources
[MetPy Documentation](https://unidata.github.io/MetPy/latest/userguide/index.html)<br>
[MetPy API reference](https://unidata.github.io/MetPy/latest/api/index.html)<br>
[Unidata's MetPy Mondays](https://www.unidata.ucar.edu/blogs/news/entry/metpy-mondays)

#### Acknowledgements
Major thanks to [Andrew Winters](https://acwinters.weebly.com/) (a UW-Madison PhD graduate!) for providing sample notebooks that helped me create the content for this lecture and this week's in-class exercises. Some lecture content is also heavily borrowed from the MetPy [getting started guide](https://unidata.github.io/MetPy/latest/userguide/startingguide.html).

# A little about MetPy

MetPy is a package used for manipulating and visualizing meteorological data. It has a number of really useful built-in functions for calculating meteorological variables and a lot of support for creating nice weather maps. It also has a number of [features similar to GEMPAK](https://unidata.github.io/MetPy/latest/userguide/gempak.html), the set of which will likely grow as MetPy development continues. It also works well with xarray!

---

Let's begin by importing a few packages that we need (we'll import stuff from MetPy later):

In [ ]:
import xarray as xr
import numpy as np
import cfgrib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

## 1. "Fun" with the GRIB file format
[GRIB](https://confluence.ecmwf.int/display/CKB/What+are+GRIB+files+and+how+can+I+read+them) files are a type of file developed by the World Meteorological Organization for transferring large streams of meteorological data, especially from weather forecasting systems. While the format is good at what it is designed for, it is much worse when utilized as a format for data storage and access. Nevertheless, the WMO seems to want to keep the grib format around, and if you've worked with any kind of forecasting data that isn't nicely packaged as netcdf files, there's a good chance you've come across the GRIB format, and you probably dislike it.

### Reading in GRIB files with xarray
Thankfully, we can usually read in grib files using xarray! All we need to do is specify the `'engine'` keyword so that `open_dataset()` knows we have a grib file. Let's try it below on a single GRIB file from the [Climate Forecast System Reanalysis](https://climatedataguide.ucar.edu/climate-data/climate-forecast-system-reanalysis-cfsr) (CFSR).

><b><font color='red'>Note:</font></b> Sometimes xarray doesn't quite work for reading GRIB files. When that happens, you might want to switch to using the [pygrib](https://pypi.org/project/pygrib/) package instead (pygrib is not currently in our class environment). Metpy Mondays has an example video for [reading GRIB files with pygrib](https://www.unidata.ucar.edu/blogs/developer/en/entry/metpy-mondays-135-reading-grib).

In [ ]:
#Read in grib
path='/share/Lecture_data/'
ds=xr.open_dataset(path+'pgbh06.gdas.1997033112.grb2',engine='cfgrib')

Yikes! Looks like that didn't quite work. That's because this particular grib file has multiple options for the vertical levels. At the end of the error message above, you can see all of the valid types of levels that you can provide as inputs to `open_dataset()` when using the cfgrib engine. Let's do what the error message tells us and use the `'filter_by_keys'` keyword to choose the type of vertical level that we want:

In [ ]:
#Read in grib with filter_by_keys
ds=xr.open_dataset(path+'pgbh06.gdas.1997033112.grb2',engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa'})

It doesn't look like it, but the data read-in actually worked. However, if you read the above warning messages, some variables were left out because they have different sets of levels associated with the `'isobaricInhPa'` level type, and xarray won't allow that. So to get around this, xarray leaves those data out. If you print your dataset below though, you can see that a lot of the data was read in correctly:

In [ ]:
ds

This is fine, but how do we know what the full set of variables is in the grib file if we can't read them all in at once with xarray?

We can do this with [cfgrib](https://pypi.org/project/cfgrib/) instead, which will give us a list of the variables associated with each `'typeOfLevel'` in the grib file (which we specified with the `filter_by_keys` argument above when we read in our data).

><b><font color='red'>Note:</font></b> This takes a minute or so to run so we'll move on to metpy and then circle back

In [ ]:
%%time
dsets=cfgrib.open_datasets(path+'pgbh06.gdas.1997033112.grb2')
dsets

---

## 2. MetPy
MetPy contains several modules that make up the package:
1. [Constants](https://unidata.github.io/MetPy/latest/api/generated/metpy.constants.html#module-metpy.constants)--a set of meteorologically relevant constants like the radius of the Earth and g.
2. [Units](https://unidata.github.io/MetPy/latest/api/generated/metpy.units.html#module-metpy.units)--MetPy uses [pint](https://pint.readthedocs.io/en/stable/) under the hood for unit-aware calculations.
3. [IO](https://unidata.github.io/MetPy/latest/api/generated/metpy.io.html#module-metpy.io)--contains functions and classes for reading in various file formats
4. [calc](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html#module-metpy.calc)--for calculating various dynamic and thermodynamic quantities!
5. [plots](https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.html#module-metpy.plots)--for making plots like Hodographs and Skew-Ts
6. [plots.ctables](https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.ctables.html#module-metpy.plots.ctables)--for working with colormaps and custom colormaps
7. [interpolate](https://unidata.github.io/MetPy/latest/api/generated/metpy.interpolate.html#module-metpy.interpolate)--various tools for data interpolation
8. [xarray](https://unidata.github.io/MetPy/latest/api/generated/metpy.xarray.html#module-metpy.xarray)--support for enhancing interoperability between metpy and xarray

Let's import a few of these:

In [ ]:
import metpy.calc as mpcalc
import metpy.plots as mplots
from metpy.units import units

### MetPy and units
MetPy requires your data to have units in order to do unit-aware calculations, and it can get rage-y when it doesn't have them (see the [units tutorial](https://unidata.github.io/MetPy/latest/tutorials/unit_tutorial.html) for further info).

#### Adding units
You can add units to data by multiplying by the unit that you want:

In [ ]:
Q_rate=2.5*units.joules/units.second
area=1*units.meters*units.meters #or 1*units('m^2')

In [ ]:
Q_rate

What is the type of these variables with units?

In [ ]:
type(Q_rate)

Then you can do unit-aware calculations with the data:

In [ ]:
#Calculate the heat flux
Qflux=Q_rate/area
Qflux

#### Converting units
You can convert units using `to()`, but in order for this to work properly your data must be a `pint.Quantity` type! Let's go through an example below:

In [ ]:
#Set up some data
a=np.arange(0,10)*units.meter
a

In [ ]:
#convert data to another unit
a=a.to('yard')
a

In [ ]:
#try using .to() on something that isn't a pint.Quantity type
np.arange(1,10).to('m')

You can also convert any units to SI units with `to_base_units()`:

In [ ]:
a=a.to_base_units()
a

#### Dealing with Temperature
Let's try to add two values that have units of ˚C:

In [ ]:
10*units.degC+5*units.degC

You get an error because the ˚C scale is an offset (it's $T_{kelvin}-273.15K$), which can make operations like the one above ambiguous. 

---

#### <font color='blue'>Exercise for the class</font>
Write a few lines of code (really just show a quick example with math) demonstrating why the offset can make calculations with temperature ambiguous:

---

To avoid this error, you want to use the `delta_degC` unit when adding/subtracting temperatures in ˚C (or the `delta_degF`  unit when adding/subtracting temperatures in ˚F!). You can also just convert all your units to Kelvin, which doesn't have this problem ;)

In [ ]:
10*units.degC+5*units.delta_degC

### Using xarray and metpy to read in data
I've saved some CFSR data as a netcdf file, which we'll use for the rest of the examples in this lecture. We can read in our netcdf data using xarray's `open_dataset`, but we need to use metpy to parse the metadata of the dataset so that it can keep track of the data's coordinate reference system (the projection the data are on!):

In [ ]:
ds=xr.open_dataset('data/cfsr.CONUS.1997033112.nc').metpy.parse_cf()
ds

### Selecting data
You can select data using xarray, or you can do it through metpy's xarray interface. However, if you want to keep track of units, then you want to use MetPy's `quantify()` method to add units to your dataset first. `quantify()` takes the units attribute of your data variables, and makes it part of the data itself:

In [ ]:
ds=ds.metpy.quantify()

In [ ]:
#Look at a single data variable
ds.t

Now let's get the zonal (u) and meridional (v) velocity at 500 hPa:

In [ ]:
#pure xarray
uwnd=ds.u.sel(isobaricInhPa=500)
vwnd=ds.v.sel(isobaricInhPa=500)
hgt=ds.gh.sel(isobaricInhPa=500)

#metpy xarray interface
#uwnd=ds.u.metpy.sel(isobaricInhPa=500)
#vwnd=ds.v.metpy.sel(isobaricInhPa=500)
#hgt=ds.gh.metpy.sel(isobaricInhPa=500)

In [ ]:
uwnd

You can use built-in xarray plotting as well:

In [ ]:
uwnd.plot()

### Calculations
[metpy.calc](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html#module-metpy.calc) has loads of useful functions that you can use to calculate dynamic and thermodynamic quantities. Some of these, like calculating derivatives, can be a pain normally, but MetPy makes it much easier!

Many of you will probably be happy to know that MetPy has a built-in function ([lat_lon_grid_deltas](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.lat_lon_grid_deltas.html#metpy.calc.lat_lon_grid_deltas)) to calculate grid cell lengths and widths in meters:

In [ ]:
dx,dy = mpcalc.lat_lon_grid_deltas(ds.longitude, ds.latitude)

In [ ]:
dx

In [ ]:
plt.pcolormesh(dx)
plt.colorbar()

#### Gradients
MetPy can also [compute gradients](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.gradient.html#metpy.calc.gradient). Let's compute dT/dy like we did in Homework 3.

><b><font color='red'>Note:</font></b> metpy's `gradient()` function returns a _tuple_ containing the computed gradient along each supplied axis!

In [ ]:
dTdy=mpcalc.gradient(ds.t,axes='y')[0]

In [ ]:
dTdy

In [ ]:
dTdy.sel(longitude=250).plot(yincrease=False)

#### Relative vorticity
The vertical component of the relative vorticity is <br>

$\zeta=\frac{\partial v}{\partial x}-\frac{\partial u}{\partial y}$

Calculating the [relative vorticity](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.vorticity.html#metpy.calc.vorticity) in MetPy is straightforward too, and it computes dx and dy under the hood for you if you use labeled xarray data: 

In [ ]:
zeta=mpcalc.vorticity(uwnd,vwnd)

In [ ]:
zeta.plot()

#### Smoothing
Smoothing is useful when your data are noisy. MetPy has a number of smoothing functions, but we'll use the [n-point smoother](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.smooth_n_point.html#metpy.calc.smooth_n_point) in this example. `smooth_n_point` takes three arguments:
1. The data to be smoothed
2. The number of points to use in smoothing (only 5 or 9 are valid for now)
3. The number of times the smoother is applied

In the example below we are using the 9-point option and are applying the smoother 10 times:

In [ ]:
hgt_smoothed=mpcalc.smooth_n_point(hgt,9,10) #data, number of points for smoothing, # of times to smooth
uwnd_smoothed=mpcalc.smooth_n_point(uwnd,9,10)
vwnd_smoothed=mpcalc.smooth_n_point(vwnd,9,10)

In [ ]:
fig=plt.figure(figsize=(12,3))
ax1=fig.add_subplot(121)
ax2=fig.add_subplot(122)
uwnd.plot(ax=ax1)
uwnd_smoothed.plot(ax=ax2)

### Making plots

#### Map plot example
Let's make a map of the 500 mbar geopotential height and the potential vorticity and winds at the same level. We'll be using cartopy to do this.
><b><font color='blue'>Note: </font></b> Here we're using matplotlibs [barbs](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.barbs.html) to add wind barbs to our plotting axis, but you can also use metpy's [BarbPlot()](https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.BarbPlot.html).

First let's set up all the plotting parameters:

In [ ]:
#Data projection and map projection
data_proj=ccrs.PlateCarree()
map_proj=ccrs.LambertConformal(central_longitude=-100,central_latitude=35, standard_parallels=(33, 45))

#Create the figure and add a subplot
fig = plt.figure(1, figsize=(14, 12))
ax = plt.subplot(111, projection=map_proj)
ax.set_extent([-100, -60, 22, 50], ccrs.PlateCarree()) #lat/lon bounds are [West,East,South,North]

# Add land, coastlines, and borders
ax.add_feature(cfeature.LAND, facecolor='0.8')
countries=cfeature.NaturalEarthFeature(category="cultural", scale="110m", 
                                       facecolor="none", name="admin_0_boundary_lines_land")
ax.add_feature(countries, linewidth=2, edgecolor="black")
ax.add_feature(cfeature.STATES.with_scale('50m'), linewidth=0.5)
ax.coastlines('50m', linewidth=0.8)

# Set up contour intervals
hgt_levs=np.arange(5000,5501,50)
zeta_levs=np.arange(5,35,2)

#Scale factor for the relative vorticity
scale=5

# #########Make the plot
# #Plot zeta (relative vorticity)
# zeta_cs=ax.contourf(ds.longitude, ds.latitude, zeta*10**scale, levels=zeta_levs, 
#                               cmap=plt.cm.YlOrRd, alpha=1, transform=data_proj,extend='max')
# #Plot the geopotential height
# gph_cs=plt.contour(ds.longitude, ds.latitude, hgt_smoothed, levels=hgt_levs, colors='navy', 
#                    linewidths=2.0, transform=data_proj)
# #label the contours
# plt.clabel(gph_cs, fmt='%d')

# # Plot wind barbs, regrid to reduce number of barbs
# ax.barbs(ds.longitude, ds.latitude, uwnd_smoothed.values, vwnd_smoothed.values, pivot='middle',
#          color='k', regrid_shape=15, transform=data_proj)

# #Add the colorbar for zeta
# cbar=plt.colorbar(zeta_cs, orientation='vertical', pad=0.03, aspect=25,shrink=0.8)
# cbar.set_label('Vorticity (10$^{–5}$ s$^{–1}$)', size='x-large',rotation=270,va='bottom')

# #Add a title
# plt.title('500-hPa Heights (m), Wind (m/s), and Rel. Vorticity (10$^{-5}$ s$^{-1}$)', loc='left');

#### Skew-Ts
For this example let's choose a single grid cell and pretend like the data at this point is from an upper air sounding. You can read more about Skew-Ts [here](https://www.noaa.gov/jetstream/upperair/skew-t-log-p-diagrams)

In [ ]:
temp=ds.t.sel(latitude=33,longitude=281) #choose a single lat/lon point for the temperature
p=ds.isobaricInhPa #pressure
rh=ds.r.sel(latitude=33,longitude=281) #relative humidity
u=ds.u.sel(latitude=33,longitude=281) #zonal velocity
v=ds.v.sel(latitude=33,longitude=281) #meridional velocity

#Calculate the dewpoint temperature
temp_dew=mpcalc.dewpoint_from_relative_humidity(temp,rh)

In [ ]:
temp_dew.metpy.units

Let's convert temperature to ˚C so we can compare it to the dewpoint temperature, which is also in ˚C. 

><b><font color='red'>Note:</font> When changing the units of a data array, we must use the metpy `convert_units()` method instead of `to()`:</b>

In [ ]:
temp=temp.metpy.convert_units(units.degC) #convert to degC
temp

Now let's make our Skew-T with the [SkewT](https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.SkewT.html) plotting command. We'll also add a Hodograph as an inset using metpy's [Hodograph()](https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.Hodograph.html) and matplotlib's [inset_axes()](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.inset_axes.html):

In [ ]:
#Make the figure
fig=plt.figure(figsize=(9, 9))
#Set up the figure for plotting a skewT
skew=mplots.SkewT(fig)

#Plot the data
skew.plot(p, temp, 'darkorange', linewidth=2) #plot the air temperature
skew.plot(p, temp_dew, 'cornflowerblue', linewidth=2) #plot the dewpoint
skew.plot_barbs(p,u,v) #add wind barbs on the side
plt.gca().set_xlim(-50,50)
plt.title('Skew-T at {}$\degree$N {}$\degree$E'.format(33,281));
plt.xlabel('T ($\degree$C)');
plt.ylabel('Pressure (hPa)');

# #add a hodograph
# from mpl_toolkits.axes_grid1.inset_locator import inset_axes #for creating insets
# #calculate windspeed
# wspd=mpcalc.wind_speed(u,v)
# axh=inset_axes(skew.ax, '50%', '50%', loc='upper right')
# h=mplots.Hodograph(axh, component_range=80.)
# h.add_grid(increment=20)
# h.plot_colormapped(u, v, wspd)  # Plot a line colored by wind speed